# Coherence tests

[L. Blackburn, Sep 2018; rewritten for Python 3.9, Dec 2022]

The purpose of this test is to check the degree of phase coherence on different baslines by calculating the amplitude loss using different coherent integration timescales. Poor phase coherence can be a result of low signal-to-noise and poor atmosphere, or an issue with ad-hoc phase correction.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
import matplotlib.pyplot as plt
import os
import sys
import seaborn as sns

sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']

nb_stdout = sys.stdout # grab for later

# helper functions
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()

def tightx(): plt.autoscale(enable=True, axis='x', tight=True)

def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')

def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])

# pd.options.display.float_format = '{:,.6f}'.format
from IPython.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [ ]:
# Define variables and load data
alist0 = 'alist.v6.2s.avg'
alist1 = 'alist.v6.30s.avg'
datadir = os.environ['DATADIR']

a0 = util.noauto(hops.read_alist(os.path.join(datadir, alist0)))
a1 = util.noauto(hops.read_alist(os.path.join(datadir, alist1)))

# Pre-process the alist files for easier manipulation
util.fix(a0)
util.add_days(a0)
util.add_path(a0)
util.add_scanno(a0)

util.fix(a1)
util.add_days(a1)
util.add_path(a1)
util.add_scanno(a1)

In [ ]:
# Compute the ratio between the amplitudes of the 30s and the 2s alist files

# Set MultiIndex columns and group by the same columns (except polarization) for averaging
idx_cols = 'expt_no source scan_id baseline polarization'.split()
group_cols = 'expt_no source scan_id baseline'.split()
a0 = a0[a0.polarization.isin({'RR', 'LL', 'YR', 'XL', 'RY', 'LX'})].set_index(idx_cols).groupby(group_cols).mean(numeric_only=True)
a1 = a1[a1.polarization.isin({'RR', 'LL', 'YR', 'XL', 'RY', 'LX'})].set_index(idx_cols).groupby(group_cols).mean(numeric_only=True)

# Compute the amplitude ratio
a0['coh'] = a1.amp / a0.amp
a = a0.reset_index().dropna()

In [ ]:
# Plot the 30s/2s amplitude ratio

plt.loglog(a.snr, a.coh, '.', ms=1);
plt.axvline(7, color='k', ls='--', lw=2, alpha=0.25);
plt.axhline(1, color='k', ls='--', lw=2, alpha=0.25);

plt.ylim(.2, 2);
plt.xlim(1, None);
plt.xlabel('2s SNR');
plt.ylabel('30s relative amplitude');
plt.title('30s/2s amplitude ratio');
wide(8, 4.5);

### Coherence loss plots by station

We make a series of plots showing the coherence loss for all baselines to a given station.

The vertical dashed lines represent the track (expt_no) boundaries.

In [ ]:
# data filters

snr_cutoff = 7 # reasonable SNR cutoff for filtering

a = a[(a.snr > snr_cutoff) & ~a.baseline.str.contains('R')]

In [ ]:
# Compute the boundaries between expt_nos
sorted_a = a.sort_values(['expt_no', 'scan_no'])
last_scans = sorted_a.groupby('expt_no')['scan_no'].max() # Find the 'max' scan_no for each expt_no
elines = (last_scans.iloc[:-1] + 0.5).to_numpy() # Drop the final expt_no and offset by 0.5

In [ ]:
# show all output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Get sorted list of unique sites from the baseline column
sites = sorted(set().union(*set(a.baseline)))

# Define cutoffs for SNR and outliers
snr_split = 50
outliers_coh = 0.8
outliers_snr = 20

for site in sites:
    df_site = a[a.baseline.str.contains(site)]

    # Plot the coherence loss for each baseline to the site
    for (bl, rows) in df_site.groupby('baseline'):
        lo_mask = rows.snr < snr_split
        hi_mask = rows.snr >= snr_split

        # Reverse the baseline if the second character is not the site
        bl = bl if bl[1] == site else bl[::-1]

        h = plt.errorbar(rows[hi_mask].scan_no, rows[hi_mask].coh, yerr=1./rows[hi_mask].snr,
                         fmt='.', label='_nolegend_' if hi_mask.sum() == 0 else bl, zorder=10)
        
        _ = plt.errorbar(rows[lo_mask].scan_no, rows[lo_mask].coh, yerr=1./rows[lo_mask].snr,
                     fmt='.', label=bl if (hi_mask.sum() == 0 and lo_mask.sum() > 0) else '_nolegend_', color=h[0].get_color(), alpha=0.5)
        
    multline(elines)

    _ = plt.title('Coherence loss in 30 seconds [%.0f MHz]' % (df_site.iloc[0].ref_freq))

    tightx()
    _ = plt.xlim(0, plt.xlim()[1]*1.1)
    _ = plt.grid(axis='y', alpha=0.25)
    _ = plt.legend(loc='best')

    outliers = df_site[(df_site.coh < outliers_coh) & (df_site.snr > outliers_snr)]
    if len(outliers) > 0:
        _ = plt.plot(outliers.scan_no, outliers.coh, 'ko', ms=8, mfc='none', mew=2, zorder=-100)
        
    wide(12, 4)
    plt.show()
    
    # Display outliers
    outliers["expt_no scan_id source baseline snr coh".split()]